### Feature Engineer dataset

This notebook requires  `train_le.csv` and `test_le.csv` datasets (requires running [dataset-01-label-encoding](./data-01-label-encoding.ipynb) first).

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./data/train_le.csv')
test = pd.read_csv('./data/test_le.csv')

In [3]:
train.head()

,Unnamed: 0,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,0,22.0,NaN,3,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,1,0,A/5 21171
1,1,38.0,C85,1,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,1,PC 17599
2,2,26.0,NaN,3,7.9250,"Heikkinen, Miss. Laina",0,3,3,0,0,1,STON/O2. 3101282
3,3,35.0,C123,3,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,0,1,1,113803
4,4,35.0,NaN,3,8.0500,"Allen, Mr. William Henry",0,5,3,1,0,0,373450


In [4]:
test.head()

,Unnamed: 0,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,892,47.0,NaN,3,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,893,3,0,1,0,363272
1,893,62.0,NaN,2,9.6875,"Myles, Mr. Thomas Francis",0,894,2,1,0,0,240276
2,894,27.0,NaN,3,8.6625,"Wirz, Mr. Albert",0,895,3,1,0,0,315154
3,895,22.0,NaN,3,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,896,3,0,1,0,3101298
4,896,14.0,NaN,3,9.2250,"Svensson, Mr. Johan Cervin",0,897,3,1,0,0,7538


In [5]:
# concat dfs
concat = train.append(test)

In [6]:
concat.shape

(1309, 13)

In [8]:
train.shape[0] + test.shape[0]

1309

Handle names first

In [13]:
NameSplit = concat.Name.str.split('[,.]')

In [14]:
NameSplit.head()

0                          [Braund,  Mr,  Owen Harris]
1    [Cumings,  Mrs,  John Bradley (Florence Briggs...
2                           [Heikkinen,  Miss,  Laina]
3     [Futrelle,  Mrs,  Jacques Heath (Lily May Peel)]
4                         [Allen,  Mr,  William Henry]
Name: Name, dtype: object

In [24]:
NameSplit.values[0]

['Braund', ' Mr', ' Owen Harris']

In [35]:
NameSplit.values[:5]

array([['Braund', ' Mr', ' Owen Harris'],
       ['Cumings', ' Mrs', ' John Bradley (Florence Briggs Thayer)'],
       ['Heikkinen', ' Miss', ' Laina'],
       ['Futrelle', ' Mrs', ' Jacques Heath (Lily May Peel)'],
       ['Allen', ' Mr', ' William Henry']], dtype=object)

In [37]:
titles = [str.strip(name[1]) for name in NameSplit.values]

In [38]:
titles[:10]

['Mr', 'Mrs', 'Miss', 'Mrs', 'Mr', 'Mr', 'Mr', 'Master', 'Mrs', 'Mrs']